In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
# from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn import neighbors
import pickle

In [17]:
match_results=pd.read_csv('All_stats.csv')

In [18]:
match_results

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,PSCH,PSCD,PSCA
0,E0,15-08-2009,Aston Villa,Wigan,0,2,A,0,1,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,E0,15-08-2009,Blackburn,Man City,0,2,A,0,1,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,E0,15-08-2009,Bolton,Sunderland,0,1,A,0,1,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,E0,15-08-2009,Chelsea,Hull,2,1,H,1,1,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,E0,15-08-2009,Everton,Arsenal,1,6,A,0,3,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4055,E0,15-02-2020,Norwich,Liverpool,0,1,A,0,0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4056,E0,16-02-2020,Aston Villa,Tottenham,2,3,A,1,2,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4057,E0,16-02-2020,Arsenal,Newcastle,4,0,H,0,0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4058,E0,17-02-2020,Chelsea,Man United,0,2,A,0,1,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
match_results.isnull().sum()

Div            0
Date           0
HomeTeam       0
AwayTeam       0
FTHG           0
            ... 
BbMxAHA     3680
BbAvAHA     3680
PSCH        3680
PSCD        3680
PSCA        3680
Length: 62, dtype: int64

In [20]:
match_results.fillna(method='pad',inplace=True)

In [21]:
match_results.isnull().sum()

Div            0
Date           0
HomeTeam       0
AwayTeam       0
FTHG           0
            ... 
BbMxAHA     3420
BbAvAHA     3420
PSCH        3420
PSCD        3420
PSCA        3420
Length: 62, dtype: int64

In [22]:
#Checking or duplicates
match_results.duplicated(keep='first').value_counts()

False    4060
dtype: int64

In [23]:
winner=[]
for i in range (len(match_results['HomeTeam'])):
    if match_results['FTHG'][i] > match_results['FTAG'][i]:
        winner.append(match_results['HomeTeam'][i])
    elif match_results['FTHG'][i] < match_results ['FTHG'][i]:
        winner.append(match_results['AwayTeam'][i])
    else:
        winner.append('Draw')
match_results['Winner'] = winner

In [24]:
#Team with most away goals
match_results['FTAG'].groupby(match_results['AwayTeam']).sum().sort_values(ascending=False).head()

AwayTeam
Man City     383
Liverpool    333
Arsenal      333
Chelsea      328
Tottenham    326
Name: FTAG, dtype: int64

In [25]:
#Team with most home goals
match_results['FTHG'].groupby(match_results['HomeTeam']).sum().sort_values(ascending=False).head()

HomeTeam
Man City      513
Chelsea       440
Liverpool     433
Arsenal       432
Man United    416
Name: FTHG, dtype: int64

In [26]:

match_results['AwayTeam'].unique()

array(['Wigan', 'Man City', 'Sunderland', 'Hull', 'Arsenal', 'Fulham',
       'Burnley', 'West Ham', 'Birmingham', 'Liverpool', 'Chelsea',
       'Wolves', 'Portsmouth', 'Man United', 'Tottenham', 'Stoke',
       'Bolton', 'Blackburn', 'Everton', 'Aston Villa', 'West Brom',
       'Blackpool', 'Newcastle', 'Norwich', 'Swansea', 'QPR',
       'Southampton', 'Reading', 'Cardiff', 'Crystal Palace', 'Leicester',
       'Watford', 'Bournemouth', 'Middlesbrough', 'Huddersfield',
       'Brighton', 'Sheffield United'], dtype=object)

In [27]:
#To get total number of shots at home and away for a team
def total_shots(team_name):
    homeshots=0
    awayshots=0
    
    if team_name in match_results['HomeTeam'].unique():
        for i in range(len(match_results['HomeTeam'])):
            if team_name==match_results['HomeTeam'][i]:
                homeshots+=match_results['HS'][i]
            elif team_name==match_results['AwayTeam'][i]:
                awayshots+=match_results['AS'][i]
        print('Team:' ,team_name, "\nHomeShots:", homeshots, "\nAwayShots:",awayshots)
    else:
        print('Check the team name')
    

In [28]:
total_shots('Tottenham')

Team: Tottenham 
HomeShots: 3540 
AwayShots: 2787


In [29]:
#Checking corr to get required features
#Corr with HomeGoals
match_results.corr()['FTHG'].sort_values()

BbAHh      -0.357771
IWH        -0.276932
BWH        -0.275366
WHH        -0.271626
B365H      -0.270561
PSCH       -0.246808
PSH        -0.245696
BbAvH      -0.242026
VCH        -0.233483
BbMxH      -0.231429
BbAv>2.5   -0.152366
BbMx>2.5   -0.152202
AS         -0.150850
HY         -0.110728
AST        -0.100651
FTAG       -0.092666
AC         -0.080305
HR         -0.077365
HTAG       -0.064765
BbOU       -0.053124
HF         -0.052330
AF         -0.048462
BbAvAHA    -0.025884
BbMxAHA    -0.023396
AY         -0.017428
BbAvAHH     0.016365
Bb1X2       0.020632
BbMxAHH     0.021031
HC          0.056891
AR          0.068981
BbAH        0.083121
BbAv<2.5    0.192708
BbMx<2.5    0.198466
PSCD        0.221174
BbMxD       0.229230
BbAvD       0.232034
VCD         0.232829
PSD         0.232894
WHD         0.252386
IWD         0.252824
BWD         0.258956
B365D       0.260831
HS          0.290248
BbMxA       0.332567
VCA         0.337363
BbAvA       0.341263
PSA         0.343503
WHA         0

In [30]:
#Corr with AwayGoals
match_results.corr()['FTAG'].sort_values()


BWA        -0.261596
IWA        -0.258477
B365A      -0.257807
WHA        -0.250446
PSCA       -0.216451
BbAvA      -0.212036
PSA        -0.210744
VCA        -0.205400
BbMxA      -0.201577
HS         -0.129916
HST        -0.109273
B365D      -0.100776
BWD        -0.100644
IWD        -0.099855
WHD        -0.093800
FTHG       -0.092666
HC         -0.091371
BbMxD      -0.081950
PSD        -0.081269
BbAvD      -0.080385
HTHG       -0.080239
VCD        -0.077048
PSCD       -0.076736
AR         -0.072254
BbMx<2.5   -0.052041
BbAv<2.5   -0.050691
BbAH       -0.036370
AF         -0.029603
BbMxAHA    -0.011439
Bb1X2      -0.009487
BbAvAHA    -0.008743
AY         -0.005372
BbOU       -0.002026
BbMxAHH    -0.001510
BbAvAHH     0.002416
BbMx>2.5    0.005128
BbAv>2.5    0.005967
HF          0.016104
AC          0.052605
HY          0.112691
HR          0.115212
BbMxH       0.241785
VCH         0.243834
BbAvH       0.250732
PSCH        0.252126
PSH         0.256607
BbAHh       0.283073
WHH         0

In [31]:
#Taking valid features into consideration
PL_matchresults=match_results[['HomeTeam','AwayTeam','FTHG','FTAG','HST','AST','B365H','B365D','B365A']]

In [32]:
PL_matchresults

,HomeTeam,AwayTeam,FTHG,FTAG,HST,AST,B365H,B365D,B365A
0,Aston Villa,Wigan,0,2,5,7,1.67,3.60,5.50
1,Blackburn,Man City,0,2,9,5,3.60,3.25,2.10
2,Bolton,Sunderland,0,1,3,13,2.25,3.25,3.25
3,Chelsea,Hull,2,1,12,3,1.17,6.50,21.00
4,Everton,Arsenal,1,6,5,9,3.20,3.25,2.30
...,...,...,...,...,...,...,...,...,...
4055,Norwich,Liverpool,0,1,1,6,11.00,6.25,1.25
4056,Aston Villa,Tottenham,2,3,4,10,4.00,3.80,1.85
4057,Arsenal,Newcastle,4,0,7,2,1.40,4.75,8.00
4058,Chelsea,Man United,0,2,1,3,1.85,3.50,4.50


In [33]:
#Regular Premier League Teams
PremierLeague_teams=['Arsenal','Aston Villa','Bournemouth','Brighton','Burnley',
                    'Chelsea','Crystal Palace','Everton','Fulham',
                    'Huddersfield','Leicester','Liverpool','Man City','Man United',
                    'Newcastle','Southampton','Swansea','Tottenham','Watford','West Ham',
                     'Wolves']

In [34]:
#Matches played by Regular Teams
PL_matchresults=PL_matchresults[PL_matchresults['HomeTeam'].isin(PremierLeague_teams) & PL_matchresults['AwayTeam'].isin(PremierLeague_teams)]


In [35]:
PL_matchresults

,HomeTeam,AwayTeam,FTHG,FTAG,HST,AST,B365H,B365D,B365A
4,Everton,Arsenal,1,6,5,9,3.20,3.25,2.30
7,Wolves,West Ham,0,2,11,13,2.63,3.30,2.70
9,Tottenham,Liverpool,2,1,11,3,3.20,3.25,2.30
13,Burnley,Man United,1,0,2,9,9.50,5.00,1.33
19,Man City,Wolves,1,0,11,2,1.33,4.50,12.00
...,...,...,...,...,...,...,...,...,...
4054,Southampton,Burnley,1,2,2,6,1.75,3.80,4.75
4056,Aston Villa,Tottenham,2,3,4,10,4.00,3.80,1.85
4057,Arsenal,Newcastle,4,0,7,2,1.40,4.75,8.00
4058,Chelsea,Man United,0,2,1,3,1.85,3.50,4.50


In [36]:
PL_matchresults.reset_index(inplace=True,drop=True)

In [37]:
PL_matchresults

,HomeTeam,AwayTeam,FTHG,FTAG,HST,AST,B365H,B365D,B365A
0,Everton,Arsenal,1,6,5,9,3.20,3.25,2.30
1,Wolves,West Ham,0,2,11,13,2.63,3.30,2.70
2,Tottenham,Liverpool,2,1,11,3,3.20,3.25,2.30
3,Burnley,Man United,1,0,2,9,9.50,5.00,1.33
4,Man City,Wolves,1,0,11,2,1.33,4.50,12.00
...,...,...,...,...,...,...,...,...,...
2246,Southampton,Burnley,1,2,2,6,1.75,3.80,4.75
2247,Aston Villa,Tottenham,2,3,4,10,4.00,3.80,1.85
2248,Arsenal,Newcastle,4,0,7,2,1.40,4.75,8.00
2249,Chelsea,Man United,0,2,1,3,1.85,3.50,4.50


In [38]:
#Rearranging columns
PL_matchresults=PL_matchresults[['HomeTeam','AwayTeam','HST','AST','B365H','B365D','B365A','FTHG','FTAG']]

In [39]:
PL_matchresults

,HomeTeam,AwayTeam,HST,AST,B365H,B365D,B365A,FTHG,FTAG
0,Everton,Arsenal,5,9,3.20,3.25,2.30,1,6
1,Wolves,West Ham,11,13,2.63,3.30,2.70,0,2
2,Tottenham,Liverpool,11,3,3.20,3.25,2.30,2,1
3,Burnley,Man United,2,9,9.50,5.00,1.33,1,0
4,Man City,Wolves,11,2,1.33,4.50,12.00,1,0
...,...,...,...,...,...,...,...,...,...
2246,Southampton,Burnley,2,6,1.75,3.80,4.75,1,2
2247,Aston Villa,Tottenham,4,10,4.00,3.80,1.85,2,3
2248,Arsenal,Newcastle,7,2,1.40,4.75,8.00,4,0
2249,Chelsea,Man United,1,3,1.85,3.50,4.50,0,2


In [40]:
#Renaming columns
PL_matchresults.rename(columns={'HST':'HomeTeam_ShotsonTarget','AST':'AwayTeam_ShotsonTarget','FTHG':'HomeGoals','FTAG':'AwayGoals'},inplace=True)

In [41]:
X=PL_matchresults.iloc[:,0:7]

In [42]:
y=PL_matchresults.iloc[:,7:9]

In [43]:
X=pd.get_dummies(X,columns=['HomeTeam','AwayTeam'])

In [44]:
X

,HomeTeam_ShotsonTarget,AwayTeam_ShotsonTarget,B365H,B365D,B365A,HomeTeam_Arsenal,HomeTeam_Aston Villa,HomeTeam_Bournemouth,HomeTeam_Brighton,HomeTeam_Burnley,...,AwayTeam_Liverpool,AwayTeam_Man City,AwayTeam_Man United,AwayTeam_Newcastle,AwayTeam_Southampton,AwayTeam_Swansea,AwayTeam_Tottenham,AwayTeam_Watford,AwayTeam_West Ham,AwayTeam_Wolves
0,5,9,3.20,3.25,2.30,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,11,13,2.63,3.30,2.70,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,11,3,3.20,3.25,2.30,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,2,9,9.50,5.00,1.33,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
4,11,2,1.33,4.50,12.00,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2246,2,6,1.75,3.80,4.75,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2247,4,10,4.00,3.80,1.85,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2248,7,2,1.40,4.75,8.00,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2249,1,3,1.85,3.50,4.50,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [46]:

rf_regressor=RandomForestRegressor(n_estimators=80,random_state=50)

In [47]:

rf_regressor.fit(X_train,y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=80, n_jobs=None, oob_score=False,
                      random_state=50, verbose=0, warm_start=False)

In [48]:
# predictions=knn_regressor.predict(X_test)
predictions=rf_regressor.predict(X_test)

In [53]:
predictions=predictions.round().astype(int)

In [54]:
predictions[120]

array([2, 1])

In [55]:
filename='Premier-League-Prediction-model.pkl'
pickle.dump(rf_regressor, open(filename, 'wb'))

In [56]:
team_1=['Burnley']
team_2=['Man City']
PL_matchresults[PL_matchresults['HomeTeam'].isin(team_1) & PL_matchresults['AwayTeam'].isin(team_2)]['B365A'].mean()



1.3516666666666666

In [57]:
PL_matchresults[PL_matchresults['HomeTeam'].isin(team_1) & PL_matchresults['AwayTeam'].isin(team_2)]['B365H'].mean()

11.416666666666666